## Homework

In [7]:
import requests 
from elasticsearch import Elasticsearch

In [2]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
documents[0].keys()

dict_keys(['text', 'section', 'question', 'course'])

In [8]:
es_client = Elasticsearch('http://localhost:9200') 

### Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text.

In [10]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

index_name = "course-questions-2"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions-2'})

In [11]:
for doc in documents:
    es_client.index(index=index_name, document=doc)

### Q3. Searching

Now let's search in our index.

We will execute a query "How do I execute a command in a running docker container?".

Use only question and text fields and give question a boost of 4, and use "type": "best_fields".

What's the score for the top ranking result?

    94.05
    84.05
    74.05
    64.05

Look at the _score field.

In [12]:
query = 'How do I execute a command in a running docker container?'

In [13]:
def elastic_search(query, filter_course=None):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }
    
    if filter_course:
        search_query["query"]["bool"]["filter"] = {
            "term": {
                "course": filter_course
            }
        }

    response = es_client.search(index=index_name, body=search_query)
    return response


In [14]:
search_results = elastic_search(query)

In [15]:
search_results.get("hits").get("max_score")

84.050095

### Q4. Filtering

Now let's only limit the questions to machine-learning-zoomcamp.

Return 3 results. What's the 3rd question returned by the search engine?

    How do I debug a docker container?
    How do I copy files from a different folder into docker container’s working directory?
    How do Lambda container images work?
    How can I annotate a graph?


In [16]:
search_results = elastic_search(query, filter_course="machine-learning-zoomcamp")

In [21]:
search_results.get("hits").get("hits")[2].get("_source").get("question")

'How do I copy files from a different folder into docker container’s working directory?'

In [24]:
search_results

ObjectApiResponse({'took': 21, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 345, 'relation': 'eq'}, 'max_score': 84.050095, 'hits': [{'_index': 'course-questions-2', '_id': 'cmC-SZABlGXZwDaNWM3A', '_score': 84.050095, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions-2', '_id': 'kWC-SZABlGXZwDaNWc0P', '_score': 51.04628, '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\

### Q5. Building a prompt

In [40]:
context_template = """
Q: {question}
A: {text}
""".strip()

context = ""

for resp in search_results['hits']['hits']:
    question = resp['_source']['question']
    ans = resp['_source']['text']

    context = context + context_template.format(question=question, text=ans) + "\n\n"

In [41]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()


prompt = prompt_template.format(question=query, context=context)

In [46]:
len(prompt)

1464

### Q6. Tokens

In [47]:
import tiktoken

In [48]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [51]:
len(encoding.encode(prompt))

323